In [151]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [152]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run /content/drive/MyDrive/UAE/Model.ipynb
%run /content/drive/MyDrive/UAE/Dataset.ipynb

plt.rcParams['font.size'] = 16
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/content


In [153]:
class Trainer:
    def __init__(self, model, optimizer, device='cpu', max_patience=20):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device

        self.max_patience = max_patience
        self.best_val_loss = float('inf')
        self.patience = 0

        self.val_cost = []
        self.train_cost = []

    def train(self, train_loader, val_loader=None, epochs=10, print_every=1, name=None):
        self.model.train()

        for epoch in range(1, epochs + 1):
            total_loss = 0.0
            for x, x_, _ in train_loader:  # label kullanılmıyor
                x = x.to(self.device)
                x_ = x_.to(self.device)

                with torch.no_grad():
                  z_hat_, _ = self.model(x_)
                z_hat, x_hat = self.model(x)

                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,64))

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            # Validation (opsiyonel)
            if val_loader:
                val_loss = self.validate(val_loader)

                # Early stopping kontrolü
                if val_loss < self.best_val_loss:
                    print('saved!')
                    torch.save(self.model, name + '.model')
                    self.best_val_loss = val_loss
                    self.patience = 0

                else:
                    self.patience = self.patience + 1

                if self.patience > self.max_patience:
                    print("Early stopping triggered.")
                    break

            if epoch % print_every == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {total_loss / len(train_loader):.6f} | Validation Loss: {val_loss:.6f}")

            self.val_cost.append(val_loss)
            self.train_cost.append(total_loss / len(train_loader))

    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0.0

        with torch.no_grad():
            for x, x_, _ in val_loader:
                x = x.to(self.device)
                x_ = x_.to(self.device)


                z_hat_, _ = self.model(x_)
                z_hat, x_hat = self.model(x)

                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,64))

                total_loss += loss.item()

        avg_loss = total_loss / len(val_loader)
        #print(f"→ Validation Loss: {avg_loss:.6f}")
        self.model.train()
        return avg_loss

In [154]:
# Custom Transform
class NoiseTransform:
    """Add some noise."""

    def __init__(self, split_ratio=0.001, dim=64):

        self.split_ratio = split_ratio
        self.dim = dim

    def __call__(self, x):
      return x + self.split_ratio * np.random.randn(self.dim,)

In [155]:
# Hyper-Parameters & Settings

batch_size = 256
lr = 0.0003

epochs = 1000
max_patience = 1000

split_ratio = 0.0001

In [156]:
# Dataset
train_dataset = DigitsDataset(mode='train', transform=NoiseTransform(split_ratio))
val_dataset = DigitsDataset(mode='val', transform=NoiseTransform(split_ratio))
test_dataset = DigitsDataset(mode='test')

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [157]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("/content/drive/MyDrive/UAE/results", exist_ok=True)

# Model
name = '/content/drive/MyDrive/UAE/results/UAE_Digits'
model = To_Uniform(
                 encoder_layers=[64, 512, 512, 512, 512, 3],
                 decoder_layers=[3, 512, 512, 512, 512, 64],
                 encoder_act=nn.SiLU,
                 decoder_act=nn.SiLU,
                 final_encoder_act=nn.Sigmoid,
                 final_decoder_act=nn.Sigmoid,
                 use_batchnorm=True
)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [158]:
# Training
trainer = Trainer(model, optimizer, device='cuda' if torch.cuda.is_available() else 'cpu', max_patience=max_patience)
trainer.train(train_loader, val_loader, epochs=epochs, name=name)

saved!
Epoch   1 | Train Loss: 0.176447 | Validation Loss: 0.170860
saved!
Epoch   2 | Train Loss: 0.162244 | Validation Loss: 0.144444
saved!
Epoch   3 | Train Loss: 0.121787 | Validation Loss: 0.092316
saved!
Epoch   4 | Train Loss: 0.083728 | Validation Loss: 0.082168
saved!
Epoch   5 | Train Loss: 0.077141 | Validation Loss: 0.078364
saved!
Epoch   6 | Train Loss: 0.075168 | Validation Loss: 0.076392
saved!
Epoch   7 | Train Loss: 0.073085 | Validation Loss: 0.074640
saved!
Epoch   8 | Train Loss: 0.071745 | Validation Loss: 0.073702
saved!
Epoch   9 | Train Loss: 0.070244 | Validation Loss: 0.072271
saved!
Epoch  10 | Train Loss: 0.068987 | Validation Loss: 0.071232
saved!
Epoch  11 | Train Loss: 0.067829 | Validation Loss: 0.069999
saved!
Epoch  12 | Train Loss: 0.066571 | Validation Loss: 0.068384
saved!
Epoch  13 | Train Loss: 0.065124 | Validation Loss: 0.066637
saved!
Epoch  14 | Train Loss: 0.063226 | Validation Loss: 0.064581
saved!
Epoch  15 | Train Loss: 0.061079 | Valida

In [159]:
# CSV dosyasına kaydet
train_losses = trainer.train_cost  # liste veya numpy array
val_losses = trainer.val_cost

np.savetxt("/content/drive/MyDrive/UAE/results/losses.csv",
           np.column_stack((train_losses, val_losses)),
           delimiter=",",
           header="train_loss,val_loss",
           comments="")

# Gift için

In [160]:
"""# Latent uzayı gift olarak kaydet

!pip install kaleido imageio

import os
import imageio.v2 as imageio
import plotly.express as px

# Klasör oluştur
os.makedirs("results/frames", exist_ok=True)

# 3D scatter plot (senin figürün)
fig = px.scatter_3d(
    df, x='x1', y='x2', z='x3',
    color=df['label'].astype(str),
    labels={'x1': 'x₁', 'x2': 'x₂', 'x3': 'x₃', 'color': 'Digit'},
    opacity=1.
)
fig.update_layout(title='Digits Dataset – 3D Latent Space')

# Kamera açılarını döndürerek frame'ler oluştur
angles = range(0, 360, 5)
images = []

for angle in angles:
    fig.update_layout(
        scene_camera=dict(
            eye=dict(x=2*np.sin(np.radians(angle)), y=2*np.cos(np.radians(angle)), z=0.5)
        )
    )
    filepath = f"results/frames/frame_{angle:03d}.png"
    fig.write_image(filepath, width=800, height=600)
    images.append(imageio.imread(filepath))

# GIF olarak kaydet
imageio.mimsave("results/digits_3d_latent_space.gif", images, fps=10)"""

'# Latent uzayı gift olarak kaydet\n\n!pip install kaleido imageio\n\nimport os\nimport imageio.v2 as imageio\nimport plotly.express as px\n\n# Klasör oluştur\nos.makedirs("results/frames", exist_ok=True)\n\n# 3D scatter plot (senin figürün)\nfig = px.scatter_3d(\n    df, x=\'x1\', y=\'x2\', z=\'x3\',\n    color=df[\'label\'].astype(str),\n    labels={\'x1\': \'x₁\', \'x2\': \'x₂\', \'x3\': \'x₃\', \'color\': \'Digit\'},\n    opacity=1.\n)\nfig.update_layout(title=\'Digits Dataset – 3D Latent Space\')\n\n# Kamera açılarını döndürerek frame\'ler oluştur\nangles = range(0, 360, 5)\nimages = []\n\nfor angle in angles:\n    fig.update_layout(\n        scene_camera=dict(\n            eye=dict(x=2*np.sin(np.radians(angle)), y=2*np.cos(np.radians(angle)), z=0.5)\n        )\n    )\n    filepath = f"results/frames/frame_{angle:03d}.png"\n    fig.write_image(filepath, width=800, height=600)\n    images.append(imageio.imread(filepath))\n\n# GIF olarak kaydet\nimageio.mimsave("results/digits_3d_la